In [1]:
# Importing DataScience libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm,skew
from scipy import stats
%matplotlib inline
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV, ElasticNetCV, LassoCV,BayesianRidge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error

In [2]:

# Load train_dataset.csv and test_dataset.csv
train_dataset = pd.read_csv('train_dataset.csv')
test_dataset = pd.read_csv('test_dataset.csv')

In [3]:
train_dataset.head()

,headline,clickbait,ID
0,"Hussein enters ""not guilty"" plea at trial",0,84698cc7-8ae2-4ea3-a425-b7091561cee6
1,Iraq peace talks draw to a close in Finland,0,a4e35ca4-15fa-43e8-b68e-91457b23afee
2,British Premier Visits Northern Ireland,0,162991ee-ea2f-41ad-a753-649a68f54311
3,The Decline of Left-Handed First Basemen,0,2cd3aa32-6ec2-4af1-bd1d-560709066b8b
4,Who Said It: Donald Trump Or Kanye West,1,72553370-c348-4603-882b-39e04b610c39


In [4]:
test_dataset.head()

,ID,headline
0,5f99b099-c4db-4a02-9753-28c5e94a6b34,Israeli military launches airstrikes into Gaza...
1,3c413552-32c0-4000-a745-b4217fe427ca,Expelled' fair use upheld
2,71060e3b-bab0-4218-b1ce-8284ae46f6c3,31 Times Frankie Boyle's Twitter Was Out Of Co...
3,f0a03121-600f-4b69-b6db-989d0f3cf28a,What Does Your Zodiac Sign Say About Your Love...
4,456f7cfa-bdfe-45bd-9e88-7c4ae53eb4ba,Larson B ice-shelf collapse reveals exotic org...


In [5]:


# The rows and columns of our dataset 
train_dataset.shape



(30400, 3)

In [6]:
# Well we have to deal with plenty of attributes 
train_dataset.columns

Index(['headline', 'clickbait', 'ID'], dtype='object')

In [7]:
# Datatype of each attribute
train_dataset.dtypes


headline     object
clickbait     int64
ID           object
dtype: object

In [8]:
# Descriptive Statistics of Numerical Variables
train_dataset.describe()

,clickbait
count,30400.000000
mean,0.499671
std,0.500008
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [9]:


# Statistics of our Categorical variables
train_dataset.describe(include=['O'])



,headline,ID
count,30400,30400
unique,23214,30400
top,Everything That Made Daniel Craig's Bond Great...,84698cc7-8ae2-4ea3-a425-b7091561cee6
freq,2,1


In [10]:
# Checking for the Missing values
# Using isnull fuction to count the total null values in each field
total = train_dataset.isnull().sum().sort_values(ascending=False) 
# Percent of missing values is estimated by dividing total missing and the original total
percent = (train_dataset.isnull().sum()/train_dataset.isnull().count()).sort_values(ascending=False)
# Concatenating the Total and Percent fields sing pandas concat fucntion
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# Displays top 20 from our max sorted list
missing_data.head(20)


,Total,Percent
headline,0,0.0
clickbait,0,0.0
ID,0,0.0


In [11]:
# Well! Things look better now! 
train_dataset.isnull().sum().max()



0

In [12]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 3.10.6 (main, Mar 10 2023, 10:55:28) [GCC 11.3.0]
NLTK: 3.8.1
Scikit-learn: 1.3.0
Pandas: 1.5.3
Numpy: 1.21.5


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier  # Import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_dataset['headline'], train_dataset['clickbait'], test_size=0.09, random_state=50)

# Convert text data to numerical features using TF-IDF
vectorizer = TfidfVectorizer(max_features=500000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Initialize and train the classification model
classification_model = RandomForestClassifier(random_state=50)
classification_model.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = classification_model.predict(X_test_tfidf)

# Evaluate the classification model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation results
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)


Accuracy: 0.9773391812865497
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.98      1372
           1       0.97      0.98      0.98      1364

    accuracy                           0.98      2736
   macro avg       0.98      0.98      0.98      2736
weighted avg       0.98      0.98      0.98      2736



In [23]:
# Load the test dataset (assuming 'test_data.csv' as the file name)
test_dataset = pd.read_csv('test_dataset.csv')

# Preprocess the test data
X_test_tfidf = vectorizer.transform(test_dataset['headline'])

# Make predictions on the test set
test_predictions = classification_model.predict_proba(X_test_tfidf)[:, 1]  # Probability of being clickbait

# Assuming you have already obtained test_predictions from your model

threshold = 0.5  # Set your threshold value here

# Convert probabilities to binary predictions based on the threshold
binary_predictions = [1 if prob >= threshold else 0 for prob in test_predictions]

# Create a DataFrame with ID and predicted probabilities
submission_df = pd.DataFrame({'ID': test_dataset['ID'], 'clickbait': binary_predictions})

# Save the predictions to a CSV file
submission_df.to_csv('submission.csv', index=False)
